In [28]:
#初始化时空立方体结构
def initSTCubInfo(config):
    splitTimeNum = config['splitTimeNum']
    splitLatNum = config['splitLatNum']
    splitLngNum = config['splitLngNum']
    panelGridNum = config['panelGridNum']
    gridWidth = config["gridWidth"]
    area = config["area"]
#     id,lat,lng,time,width
    STCubInfo = []
    for t in range(splitTimeNum):
        for i in range(splitLatNum):
            for j in range(splitLngNum):
                STCubInfo.append({"id":panelGridNum*t+i*splitLngNum+j,"lat":area["minLat"]+gridWidth*i,"lng":area["minLng"]+gridWidth*j,"time":t})  
    return STCubInfo

In [29]:
# def getValue():
#     if testArea['minLat']==0 or point["latitude"]<testArea['minLat']:
#             print(carNo,point)
#             testArea['minLat'] = point["latitude"]
#         if point["latitude"]>testArea['maxLat']:
#             testArea['maxLat'] = point["latitude"]
        
#         if testArea['minLng']==0 or point["longitude"]<testArea['minLng']:
#             print(carNo,point)
#             testArea['minLng'] = point["longitude"]
#         if point["longitude"]>testArea['maxLng']:
#             testArea['maxLng'] = point["longitude"]
# 根据id更新时空立方体
def updateSTCub(STCub,gridID,trajID,order,points):
    if gridID not in STCub.keys():
        STCub[gridID] = {}#trajID,points
    if trajID not in STCub[gridID].keys():
        STCub[gridID][trajID] = {}
    STCub[gridID][trajID][order] = points

# 根据轨迹点的经纬度和时间更新轨迹的包围盒bbx
def updateBBX(bbx,point):
    if bbx['timeRange']['min'] is None or bbx['timeRange']['min']>point['time']:
        bbx['timeRange']['min'] = point['time']
    if bbx['timeRange']['max'] is None or bbx['timeRange']['max']<point['time']:
        bbx['timeRange']['max'] = point['time']
    if bbx['areaRange']['minLat']==0.0 or bbx['areaRange']['minLat']>point['latitude']:
        bbx['areaRange']['minLat']= point['latitude']
    if bbx['areaRange']['maxLat']<point['latitude']:
        bbx['areaRange']['maxLat']= point['latitude']
    if bbx['areaRange']['minLng']==0.0 or bbx['areaRange']['minLng']>point['longitude']:
        bbx['areaRange']['minLng']= point['longitude']
    if bbx['areaRange']['maxLng']<point['longitude']:
        bbx['areaRange']['maxLng']= point['longitude']

        
def updateTrajSTCubs(lastGridID,stcubes):
    stcubes.add(lastGridID)
def inRange(area,point):
    return point['longitude']>area["minLng"] and point['longitude']<area["maxLng"] and point["latitude"]>area["minLat"] and point["latitude"]<area["maxLat"]

def getSDATA(carNo,trajsSDATA):
    if carNo not in trajsSDATA:
        trajsSDATA[carNo] = {
        'stcubes':set(),
        "bbx":{
            "timeRange":{"min":None,"max":None},
            "areaRange":{
                "minLat" : 0.0,
                "maxLat" : 0.0,
                "minLng" : 0.0,
                "maxLng" : 0.0
            }
        },
        "order":0
        }
    return trajsSDATA[carNo]
def splitTraj(traj,STCub,config,trajsSDATA):
    points = []
    lastGridID = None
#     order = 0
    if DataSource=="taxi":
        carNo = traj["carNo"]
    else:
        if DataSource=='phone':
            carNo = traj["IMEI"]
    SDATA = getSDATA(carNo,trajsSDATA)
#     轨迹的包围盒，
# 格式：{
#     timeRange:{min:string,max:string},
#     areaRange:{
#         "minLat" : float,
#         "maxLat" : float,
#         "minLng" : float,
#         "maxLng" : float
#     }
# }
    for point in traj["points"]:
#             更新轨迹的包围盒bbx
        if inRange(config["area"],point)==False:
            print(carNo,point)
        updateBBX(SDATA["bbx"],point)
        lngIndex = math.floor((point["longitude"]-config["area"]["minLng"])/config["gridWidth"])
        latIndex = math.floor((point["latitude"]-config["area"]["minLat"])/config["gridWidth"])
        time = [int(x) for x in point["time"].split(' ')[1].split(':')[0:2]]
        timeIndex = math.floor((time[0]*60+time[1])/config["splitInterval"])
        gridID = config["panelGridNum"]*timeIndex+latIndex*config["splitLngNum"]+lngIndex       
#         当前点对应的网格不同，将当前points存入之前的网格
        if lastGridID is None:
            lastGridID = gridID
        if lastGridID!=gridID:
            #             插入网格内
            if len(points)>0:
                updateSTCub(STCub,lastGridID,carNo,SDATA["order"],points)
#                 更新轨迹经过的时空立方体id集合
                updateTrajSTCubs(lastGridID,SDATA["stcubes"])
                SDATA["order"]+=1
            lastGridID = gridID
            points = []
        points.append(point)
    if len(points)>0:
        updateSTCub(STCub,lastGridID,carNo,SDATA["order"],points)
        SDATA["order"]+=1
        updateTrajSTCubs(lastGridID,SDATA["stcubes"])
#     trajsSDATA[carNo] = {
#         'stcubes':stcubes,
#         "bbx":bbx
#     }
# 读取指定路径的轨迹文件，并将数据存入STCube
def Trajs2STCube(filePath,STCub,trajsSDATA):
    # 对出租车文件中的轨迹数据存入时空立方体
    with open(filePath,'r',encoding='utf8')as fp:
        json_data = json.load(fp)
        print("轨迹数目",len(json_data))
        for carInfo in json_data:
            splitTraj(carInfo,STCub,config,trajsSDATA)

In [30]:
def write_json_to_file(json_obj,filename):
    with open(filename, 'w') as outfile:
        json.dump(json_obj, outfile)
    print("文件写入完成")

In [31]:
import math
import json
import util

DataSource = "phone"#taxi,phone
# 是否需要初始化时空立方体的划分信息，初次执行需要初始化
initSTCube = False
FILES_DIR = {
    # 文件夹
    # taxi文件在../data_in_use/taxi.json,phone文件是一组划分完时间片的文件，所以是../originPhoneData/下的所有文件
    "taxiFilePath": "../originPhoneData/",
    # "taxiFilePath": "../data_in_use/taxi_1000.json",
}

SERVER_PATH = "../tempData/"
writePath = {
    "taxiSDATAFilePath" : SERVER_PATH + DataSource +"SDATA.json",
    "taxiSTCFilePath" : SERVER_PATH + DataSource + "STCube.json",
    "STCubePath": SERVER_PATH + "STCube.json",
    "STCubeConfigPath": SERVER_PATH + "STCubeConfig.json"
}
# target = "taxiSTCFilePath"


In [32]:
# 导入时空立方体的配置项，并根据initSTCube的决定是否初始化时空立方体基本信息

# 划分时空立方体的分辨率
config = util.getJsonData('./config.json')
# config:
#     splitLngNum:int #横向 划分的网格数目
#     splitInterval:int #将一天划分的时间段长度，单位分钟，默认十分钟一个时间段
# area:{
#         "minLat" : float,
#         "maxLat" : float,
#         "minLng" : float,
#         "maxLng" : float
#     }

    # # 地理平面被划分成的网格数目
config["panelGridNum"] = config["splitLatNum"]*config["splitLngNum"]

if initSTCube:
    # 计算得到的属性值
    config["gridWidth"] = (config["area"]["maxLng"]-config["area"]["minLng"])/config["splitLngNum"]
    config["splitLatNum"] = math.ceil((config["area"]["maxLat"]-config["area"]["minLat"])/config["gridWidth"])
    config["splitTimeNum"] = math.ceil(1440/config["splitInterval"])
    # # 地理平面被划分成的网格数目
    config["panelGridNum"] = config["splitLatNum"]*config["splitLngNum"]
    # 保存创建stcube时的配置项信息
    write_json_to_file(config,writePath["STCubeConfigPath"])
    # 更新当前的config文件
    # write_json_to_file(config,'./config.json')

    # 初始化时空立方体,并将基本信息存入文件
    STCubInfo = initSTCubInfo(config)
    write_json_to_file(STCubInfo,writePath["STCubePath"])

In [33]:
# 出租车轨迹数据构建时空立方体

if DataSource == "taxi": 
    print("出租车轨迹数据构建时空立方体")
    STCub = {}
    trajsSDATA = {}
    Trajs2STCube(FILES_DIR["taxiFilePath"],STCub,trajsSDATA)
    write_json_to_file(STCub, writePath['taxiSTCFilePath'])
    for d in trajsSDATA:
        SDATA = trajsSDATA[d]
        SDATA["stcubes"] = list(SDATA["stcubes"])
        SDATA["stcubes"].sort()
    write_json_to_file(trajsSDATA, writePath["taxiSDATAFilePath"])
    print("出租车轨迹数据构建时空立方体 完成!!!!!")

In [34]:
#手机基站轨迹数据创建STCube

if DataSource == "phone": 
    print("手机基站轨迹数据构建时空立方体 开始!!!!!")
    files = util.list_files(FILES_DIR["taxiFilePath"])
    STCub = {}
    trajsSDATA = {}
    for file in files:
        # 去除掉前一天的数据
        if file=="TOP10000mpt2014114 1140010sortbyid.txt":
            continue
        print("file",FILES_DIR["taxiFilePath"]+file)
        Trajs2STCube(FILES_DIR["taxiFilePath"]+file,STCub,trajsSDATA)
    write_json_to_file(STCub, writePath['taxiSTCFilePath'])
    for d in trajsSDATA:
        SDATA = trajsSDATA[d]
        SDATA["stcubes"] = list(SDATA["stcubes"])
        SDATA["stcubes"].sort()
    write_json_to_file(trajsSDATA, writePath["taxiSDATAFilePath"])
    print("手机基站轨迹数据构建时空立方体 完成!!!!!")
#         "carNo"
#         "points"


手机基站轨迹数据构建时空立方体 开始!!!!!
file ../originPhoneData/TOP10000mpt2014114 1140020sortbyid.txt
轨迹数目 2517
file ../originPhoneData/TOP10000mpt2014114 1140030sortbyid.txt
轨迹数目 2600
file ../originPhoneData/TOP10000mpt2014114 1140040sortbyid.txt
轨迹数目 2550
file ../originPhoneData/TOP10000mpt2014114 1140050sortbyid.txt
轨迹数目 0
file ../originPhoneData/TOP10000mpt2014114 1140060sortbyid.txt
轨迹数目 2041
file ../originPhoneData/TOP10000mpt2014114 1140110sortbyid.txt
轨迹数目 2499
file ../originPhoneData/TOP10000mpt2014114 1140120sortbyid.txt
轨迹数目 2454
file ../originPhoneData/TOP10000mpt2014114 1140130sortbyid.txt
轨迹数目 2403
file ../originPhoneData/TOP10000mpt2014114 1140140sortbyid.txt
轨迹数目 2363
file ../originPhoneData/TOP10000mpt2014114 1140150sortbyid.txt
轨迹数目 2354
file ../originPhoneData/TOP10000mpt2014114 1140160sortbyid.txt
轨迹数目 2300
file ../originPhoneData/TOP10000mpt2014114 1140210sortbyid.txt
轨迹数目 2309
file ../originPhoneData/TOP10000mpt2014114 1140220sortbyid.txt
轨迹数目 2263
file ../originPhoneData/TOP100